# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Experiment, ScriptRunConfig, Workspace
from azureml.train.hyperdrive import BanditPolicy, HyperDriveConfig, PrimaryMetricGoal, RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.train.sklearn import SKLearn

from compute_cluster import get_compute_cluster

from IPython.display import display

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:

ws = Workspace.from_config()
experiment_name = 'udacity-project-hyperparameter-tuning'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [3]:

compute_target = get_compute_cluster(ws)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
        "--C":uniform(0.01,1),
        "--max_iter": choice(10, 20, 50, 200)
})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(
    source_directory='./',
    compute_target=compute_target,
    entry_script='train.py'
)

hyperdrive_run_config = HyperDriveConfig(
    estimator=estimator,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=50
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [5]:
from azureml.widgets import RunDetails
#TODO: Submit your experiment
hyper_drive_run = experiment.submit(hyperdrive_run_config)



## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(hyper_drive_run).show()

hyper_drive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_1165e572-021e-4c03-96ea-fe3101cf98f2
Web View: https://ml.azure.com/runs/HD_1165e572-021e-4c03-96ea-fe3101cf98f2?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-227003/workspaces/quick-starts-ws-227003&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-03-01T15:35:50.412717][GENERATOR][INFO]Trying to sample '50' jobs from the hyperparameter space
[2023-03-01T15:35:51.2974482Z][SCHEDULER][INFO]Scheduling job, id='HD_1165e572-021e-4c03-96ea-fe3101cf98f2_0' 
[2023-03-01T15:35:51.3471133Z][SCHEDULER][INFO]Scheduling job, id='HD_1165e572-021e-4c03-96ea-fe3101cf98f2_1' 
[2023-03-01T15:35:51.5626155Z][SCHEDULER][INFO]Scheduling job, id='HD_1165e572-021e-4c03-96ea-fe3101cf98f2_2' 
[2023-03-01T15:35:51.6665335Z][SCHEDULER][INFO]Scheduling job, id='HD_1165e572-021e-4c03-96ea-fe3101cf98f2_3' 
[2023-03-01T15:35:51.6341044Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_1165e572-021e-4c03-96ea-fe3101cf98f2_

{'runId': 'HD_1165e572-021e-4c03-96ea-fe3101cf98f2',
 'target': 'project-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-03-01T15:35:49.885183Z',
 'endTimeUtc': '2023-03-01T15:44:01.240539Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '6bd93c69-44c2-4ebf-bfd5-0f9d15cf08a6',
  'user_agent': 'python/3.8.10 (Linux-5.15.0-1031-azure-x86_64-with-glibc2.17) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': 'infinite_space_size',
  'score': '0.9078947368421053',
  'best_child_run_id': 'HD_1165e572-021e-4c03-96ea-fe3101cf98f2_28',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_1165e572-021e-4c03-96ea-fe3101cf98f2_28'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'teleme

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [14]:
import json
best_run = hyper_drive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run')
display(best_run)
print('Parameters')
print(json.dumps(best_run_metrics, indent=2))

Best Run


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-hyperparameter-tuning,HD_1165e572-021e-4c03-96ea-fe3101cf98f2_28,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


Parameters
{
  "Regularization Strength:": 0.5427355750424361,
  "Max iterations:": 200,
  "Accuracy": 0.9078947368421053
}


In [17]:
#TODO: Save the best model
model = best_run.register_model(model_name='hyper_drive_best_model', model_path='./outputs/model.pkl', properties=dict(best_run_metrics))

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/model.pkl in the set of files uploaded to the run: ['system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path outputs/model.pkl in the set of files uploaded to the run: ['system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', '

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

